# En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [12]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
from datetime import datetime, timedelta
pd.options.display.max_columns=None

df_attacks= pd.read_csv('../Datos/attacks_limpieza_completa.csv', index_col= 0)
df_clima= pd.read_csv('../Datos/clima_paises.csv')

In [13]:
df_paises = df_attacks[df_attacks.country.isin(['usa', 'australia', 'new zealand', 'south africa', 'papua new guinea'])]

Definimos las variables que vamos a necesitar para hacer las llamadas a cada uno de los métodos de la clase. 

In [14]:
dicc= {'usa': [39.7837304, -100.445882], 'australia': [-24.7761086,  134.755], 'South Africa': [-28.8166236, 24.991639], 'New Zealand': [-41.5000831, 172.8344077], 'Papua New Guinea':[-5.6816069, 144.2489081] }

Creamos la clase

In [ ]:

   
def llamada_api(dicc):

 df_todo = pd.DataFrame()
        for k, v in dicc.items():
            url =  f'http://www.7timer.info/bin/api.pl?lon={v[1]}5&lat=-{v[0]}6&product=meteo&output=json'
            response=requests.get(url=url)
            df = pd.json_normalize(response.json()['dataseries'])
            df["country"] = k
            df_todo = pd.concat([df, df_todo], axis = 0)
        
        
        #castear un string que dentro tiene diccionarios
        
    df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

    df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)
    
# cada columna será la key del diccionario
x = df_clima['rh_profile'].apply(pd.Series) 
x[0].apply(pd.Series)['layer'][0]
x


#desempaquetado1

for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)['layer'][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)
    
    
    
    
y = df_clima['wind_profile'].apply(pd.Series) 
y[0].apply(pd.Series)['layer'][0]
y


for i in range(len(y.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wi_speed" + str(y[i].apply(pd.Series)['layer'][0]) 
    valores = list(y[i].apply(pd.Series)["speed"])
    df_clima.insert(i, nombre, valores)
    
    nombre2 = "wi_direction" + str(y[i].apply(pd.Series)['layer'][0]) 
    valores2 = list(y[i].apply(pd.Series)["direction"])
    df_clima.insert(i, nombre2, valores2)

#group by
df_mean = df_clima.groupby("country").mean()
df_mean

# merge 
df_final = pd.merge(left = df_paises, right= df_mean, how= "left", left_on= "country", right_on= "country")
df_final

#guardado

df_final.to_csv('../Datos/etl_transformacion.csv', index=False)

In [ ]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, lat, lon, ciudad):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 
        self.lat = lat
        self.lon = lon
        self.ciudad = ciudad

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def llamada_API(self, producto):
        
        # producto es una variable nueva, por lo que lo primero que tenemos que hacer es definirla
        self.producto = producto
    
        # hacemos la llamada  a la API
        # Tendremos que poner el self delante de long y lat ya que son variables definidas en el constructor. El self podríamos decir que es como el camino de baldosas amarillas que indica al método de donde tiene que sacar esas variables
        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.lon}&lat={self.lat}&product={producto}&output=json'

        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

        # convertimos los resultados en un dataframe: 
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        return df
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_civil(self, df): 

        # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
        # En este caso será la fecha del día de hoy. 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        # de nuevo usamos el self para crear las nuevas columnas en función de los parámetros pasados al definir la clase. Recordamos, el caminito de baldosas amarillas.
        df["latitud"] = self.lat
        df["longitud"] = self.lon
        df["ciudad"] = self.ciudad
        return df
    
    
    # creamos una clase para limpiar el resultado de la llamada a la API para el producto astro
    def limpiar_astro(self, df):

        #seleccionamos solo las columnas que nos interesan
        df = df[["seeing", "transparency", "timepoint"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["ciudad"] = self.ciudad

        return df
    
    # de nuevo, hacemos lo mismo que en la lección anterior. 
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad): 

        df_hoy = pd.merge(df_civil , df_visibilidad , on=['fecha', "timepoint", "ciudad"], how = "inner")
        print("El df de hoy es ", df_hoy.columns)
        print("-----------------------------------------")
        print("El df de completo es ", df_completo.columns)
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_hoy], axis = 0)

        # ahora es el turno de unir el dataframe con la información de la visibilidad con el completo
        # en este caso el how lo ponemos como left ya que queremos que se quede con toda la info que en la primera tabla que le pasamos que es la que tiene toda la información
        #df_completo = pd.merge(df_completo , df_visibilidad , on=['fecha', "timepoint"], how = "inner")

        # guardamos los datos
        df_completo.to_pickle('datos/datos_actualizados.pkl')
        df_completo.to_csv('datos/datos_actualizados.csv')

        return df_completo
    
    

    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)

    def limpiar_dataframe(self, df, lista_columnas): 

        #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        # quitar columnas repetidas:
        
        df.drop(["ciudad_y"], axis = 1, inplace = True)
        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('datos/datos_Madrid.pkl')
        df.to_csv('datos/datos_Madrid.csv')

        return df